In [ ]:
# ============================================================================
# FRAUD DETECTION EDA: Understanding the Elliptic Dataset
# ============================================================================
# 
# LEARNING GOALS:
# 1. Understand data structure (nodes, edges, features, labels)
# 2. Identify class imbalance and its implications
# 3. Analyze graph topology (degree distribution, connectivity)
# 4. Discover temporal patterns (fraud evolves over time)
# 5. Inform model design decisions
#
# INTERVIEW TIP:
# "Before building any model, I always perform thorough EDA to understand
# the data generating process, identify potential pitfalls, and validate
# modeling assumptions."
# ============================================================================

import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from pathlib import Path
from collections import Counter

# Set project root
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
    
from src.data.download import EllipticDataLoader
from src.utils.config import get_config

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# ============================================================================
# STEP 1: LOAD DATA
# ============================================================================

print("=" * 80)
print("LOADING ELLIPTIC DATASET")
print("=" * 80)

config = get_config()
loader = EllipticDataLoader(config.data.raw_data_dir / "elliptic")

# Load the three CSV files
features_df, edges_df, classes_df = loader.load()

print("\n✓ Data loaded successfully!")

# ============================================================================
# STEP 2: BASIC DATA INSPECTION
# ============================================================================

print("\n" + "=" * 80)
print("DATA STRUCTURE INSPECTION")
print("=" * 80)

print("\n1. FEATURES DATAFRAME")
print(f"   Shape: {features_df.shape}")
print(f"   Columns: {features_df.columns.tolist()[:10]}... (showing first 10)")
print(f"   Memory usage: {features_df.memory_usage(deep=True).sum() / 1e6:.2f} MB")
print("\n   First few rows:")
print(features_df.head(3))

# EXPLANATION:
# Column 0: Transaction ID (unique identifier)
# Columns 1-94: Local features (transaction-specific attributes)
# Columns 95-166: Aggregate features (statistics of connected transactions)
# Column 167: Time step (1-49, representing time period)

print("\n2. EDGES DATAFRAME")
print(f"   Shape: {edges_df.shape}")
print(f"   Column names: {edges_df.columns.tolist()}")
print("\n   First few rows:")
print(edges_df.head(3))

# EXPLANATION:
# txId1 → txId2: Directed edge (money flows from txId1 to txId2)
# This represents Bitcoin transactions where outputs of one become inputs of another

print("\n3. CLASSES DATAFRAME")
print(f"   Shape: {classes_df.shape}")
print(f"   Column names: {classes_df.columns.tolist()}")
print("\n   First few rows:")
print(classes_df.head(3))

# EXPLANATION:
# txId: Transaction ID
# class: "unknown", "1" (licit), "2" (illicit)

# ============================================================================
# STEP 3: CLASS DISTRIBUTION ANALYSIS
# ============================================================================

print("\n" + "=" * 80)
print("CLASS DISTRIBUTION (THE MOST CRITICAL EDA STEP)")
print("=" * 80)

# Merge classes with features to get time information
features_df.columns = (
    ['txId']
    + [f'feature_{i}' for i in range(1, 166)]  # 165 features
    + ['time_step']
)

classes_df.columns = ['txId', 'class']

data = features_df.merge(classes_df, on='txId', how='left')

# Count classes
class_counts = data['class'].value_counts()
total = len(data)

print("\nClass Distribution:")
for cls, count in class_counts.items():
    print(f"  {cls:10s}: {count:6d} ({count/total*100:5.2f}%)")

# CRITICAL INSIGHT:
# ~77% unlabeled → Semi-supervised learning opportunity
# ~2% fraud → Extreme class imbalance → Need weighted loss or sampling

print("\n⚠️  KEY OBSERVATION:")
print("   Class imbalance ratio: 1:20 (fraud:licit)")
print("   → MUST use weighted CrossEntropyLoss or Focal Loss")
print("   → Evaluation metrics: Precision, Recall, F1 (not accuracy!)")

# ============================================================================
# STEP 4: VISUALIZE CLASS DISTRIBUTION
# ============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart
labeled_data = data[data['class'] != 'unknown']
class_counts_labeled = labeled_data['class'].value_counts()

axes[0].pie(
    class_counts_labeled.values, 
    labels=['Licit', 'Illicit'], 
    autopct='%1.1f%%',
    colors=['#2ecc71', '#e74c3c'],
    startangle=90
)
axes[0].set_title('Labeled Data Distribution\n(Excluding Unknown)', fontsize=14, fontweight='bold')

# Bar chart (all classes)
class_counts.plot(kind='bar', ax=axes[1], color=['#95a5a6', '#2ecc71', '#e74c3c'])
axes[1].set_title('Full Dataset Class Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Class', fontsize=12)
axes[1].set_ylabel('Count', fontsize=12)
axes[1].set_xticklabels(['Unknown', 'Licit', 'Illicit'], rotation=0)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('class_distribution.png', dpi=150, bbox_inches='tight')
print("\n✓ Saved: class_distribution.png")

# ============================================================================
# STEP 5: TEMPORAL ANALYSIS
# ============================================================================

print("\n" + "=" * 80)
print("TEMPORAL ANALYSIS (Fraud Patterns Over Time)")
print("=" * 80)

# Fraud ratio over time
temporal_stats = data.groupby('time_step')['class'].apply(
    lambda x: (x == '2').sum() / ((x == '1').sum() + (x == '2').sum())
).reset_index()
temporal_stats.columns = ['time_step', 'fraud_ratio']

print("\nFraud Ratio by Time Step:")
print(temporal_stats.head(10))

# Plot
plt.figure(figsize=(14, 5))
plt.plot(temporal_stats['time_step'], temporal_stats['fraud_ratio'] * 100, 
         marker='o', linewidth=2, markersize=6, color='#e74c3c')
plt.axhline(y=temporal_stats['fraud_ratio'].mean() * 100, 
            linestyle='--', color='gray', label=f'Mean: {temporal_stats["fraud_ratio"].mean()*100:.2f}%')
plt.title('Fraud Ratio Over Time Steps', fontsize=14, fontweight='bold')
plt.xlabel('Time Step', fontsize=12)
plt.ylabel('Fraud Ratio (%)', fontsize=12)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('temporal_fraud_ratio.png', dpi=150, bbox_inches='tight')
print("\n✓ Saved: temporal_fraud_ratio.png")

# INSIGHT:
print("\n⚠️  KEY OBSERVATION:")
print("   Fraud ratio varies over time → Temporal features are important!")
print("   In production: Use time-based validation (not random split)")

# ============================================================================
# STEP 6: GRAPH STRUCTURE ANALYSIS
# ============================================================================

print("\n" + "=" * 80)
print("GRAPH TOPOLOGY ANALYSIS")
print("=" * 80)

# Build NetworkX graph for analysis
print("\nBuilding NetworkX graph (this may take ~30 seconds)...")
G = nx.DiGraph()
G.add_edges_from(edges_df.values)

print(f"✓ Graph built: {G.number_of_nodes():,} nodes, {G.number_of_edges():,} edges")

# Basic graph statistics
print("\nGraph Statistics:")
print(f"  Nodes: {G.number_of_nodes():,}")
print(f"  Edges: {G.number_of_edges():,}")
print(f"  Density: {nx.density(G):.6f}")

# Degree distribution
in_degrees = dict(G.in_degree())
out_degrees = dict(G.out_degree())

print(f"\nDegree Statistics:")
print(f"  Avg in-degree:  {np.mean(list(in_degrees.values())):.2f}")
print(f"  Avg out-degree: {np.mean(list(out_degrees.values())):.2f}")
print(f"  Max in-degree:  {max(in_degrees.values())}")
print(f"  Max out-degree: {max(out_degrees.values())}")

# INSIGHT:
print("\n⚠️  KEY OBSERVATION:")
print("   Some nodes have VERY high degree → Potential hubs (exchanges?)")
print("   → GraphSAGE's neighbor sampling helps handle this!")

# ============================================================================
# STEP 7: VISUALIZE DEGREE DISTRIBUTION
# ============================================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# In-degree distribution (log scale)
in_deg_counts = Counter(in_degrees.values())
degrees, counts = zip(*sorted(in_deg_counts.items()))

axes[0].loglog(degrees, counts, 'o-', color='#3498db', alpha=0.7)
axes[0].set_title('In-Degree Distribution (Log-Log)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('In-Degree', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].grid(alpha=0.3)

# Out-degree distribution
out_deg_counts = Counter(out_degrees.values())
degrees, counts = zip(*sorted(out_deg_counts.items()))

axes[1].loglog(degrees, counts, 'o-', color='#e74c3c', alpha=0.7)
axes[1].set_title('Out-Degree Distribution (Log-Log)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Out-Degree', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('degree_distribution.png', dpi=150, bbox_inches='tight')
print("\n✓ Saved: degree_distribution.png")

# INSIGHT:
print("\n⚠️  GRAPH STRUCTURE INSIGHT:")
print("   Power-law distribution → Real-world network (not random graph)")
print("   Few high-degree hubs, many low-degree nodes")
print("   → This is WHY GNNs work! Structure contains information.")

# ============================================================================
# STEP 8: ANALYZE FRAUD VS LICIT NODE CHARACTERISTICS
# ============================================================================

print("\n" + "=" * 80)
print("FRAUD vs LICIT: NETWORK CHARACTERISTICS")
print("=" * 80)

# Get degree for labeled nodes
node_degrees = pd.DataFrame({
    'txId': list(in_degrees.keys()),
    'in_degree': list(in_degrees.values()),
    'out_degree': list(out_degrees.values())
})

# Merge with classes
node_analysis = node_degrees.merge(classes_df, on='txId', how='inner')
node_analysis = node_analysis[node_analysis['class'] != 'unknown']

# Compare degrees
fraud_degrees = node_analysis[node_analysis['class'] == '2']
licit_degrees = node_analysis[node_analysis['class'] == '1']

print("\nDegree Statistics by Class:")
print("\nFraud (Illicit) Nodes:")
print(f"  Avg in-degree:  {fraud_degrees['in_degree'].mean():.2f}")
print(f"  Avg out-degree: {fraud_degrees['out_degree'].mean():.2f}")
print(f"  Median in-degree:  {fraud_degrees['in_degree'].median():.2f}")
print(f"  Median out-degree: {fraud_degrees['out_degree'].median():.2f}")

print("\nLicit (Legitimate) Nodes:")
print(f"  Avg in-degree:  {licit_degrees['in_degree'].mean():.2f}")
print(f"  Avg out-degree: {licit_degrees['out_degree'].mean():.2f}")
print(f"  Median in-degree:  {licit_degrees['in_degree'].median():.2f}")
print(f"  Median out-degree: {licit_degrees['out_degree'].median():.2f}")

# Statistical test
from scipy.stats import mannwhitneyu

stat, p_value = mannwhitneyu(
    fraud_degrees['in_degree'], 
    licit_degrees['in_degree']
)

print(f"\nMann-Whitney U Test (In-Degree):")
print(f"  Statistic: {stat:.2f}")
print(f"  P-value: {p_value:.4f}")
if p_value < 0.05:
    print("  ✓ Fraud and licit nodes have SIGNIFICANTLY different degree distributions!")
else:
    print("  ✗ No significant difference found.")

# ============================================================================
# STEP 9: VISUALIZE FEATURE DISTRIBUTIONS
# ============================================================================

print("\n" + "=" * 80)
print("FEATURE ANALYSIS")
print("=" * 80)

# Select a few features for visualization
sample_features = ['feature_1', 'feature_2', 'feature_10', 'feature_50']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

labeled_data_with_features = data[data['class'] != 'unknown']

for idx, feature in enumerate(sample_features):
    fraud_vals = labeled_data_with_features[labeled_data_with_features['class'] == '2'][feature]
    licit_vals = labeled_data_with_features[labeled_data_with_features['class'] == '1'][feature]
    
    axes[idx].hist(licit_vals, bins=50, alpha=0.6, label='Licit', color='#2ecc71', density=True)
    axes[idx].hist(fraud_vals, bins=50, alpha=0.6, label='Fraud', color='#e74c3c', density=True)
    axes[idx].set_title(f'{feature} Distribution', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Value', fontsize=10)
    axes[idx].set_ylabel('Density', fontsize=10)
    axes[idx].legend()
    axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('feature_distributions.png', dpi=150, bbox_inches='tight')
print("\n✓ Saved: feature_distributions.png")

# ============================================================================
# STEP 10: SUMMARY & MODELING IMPLICATIONS
# ============================================================================

print("\n" + "=" * 80)
print("🎯 EDA SUMMARY & MODELING DECISIONS")
print("=" * 80)

print("""
KEY FINDINGS:
✓ Severe class imbalance (1:20 fraud:licit ratio)
✓ Temporal patterns exist (fraud ratio varies over time)
✓ Power-law degree distribution (real-world network)
✓ Fraud nodes have different network characteristics

IMPLICATIONS FOR MODEL DESIGN:
1. Loss Function:
   → Use weighted CrossEntropyLoss (fraud_weight = 10-20)
   → Alternative: Focal Loss for hard examples

2. Evaluation Metrics:
   → PRIMARY: Precision, Recall, F1-Score, AUC-ROC
   → AVOID: Accuracy (misleading with 2% fraud)
   
3. Train/Val/Test Split:
   → Time-based split (NOT random)
   → Train on time steps 1-35, Val on 36-42, Test on 43-49
   → Simulates real deployment (predict future fraud)

4. Model Architecture:
   → GraphSAGE with 2 layers (2-hop neighborhood)
   → Neighbor sampling (10, 5) to handle high-degree nodes
   → Dropout 0.5 for regularization

5. Feature Engineering:
   → Keep all 166 features initially
   → Later: Feature importance analysis
   → Consider adding: degree features, clustering coefficient

6. Semi-Supervised Learning:
   → 77% unlabeled data → Use self-training or pseudo-labeling
   → Advanced: Label propagation on graph

NEXT STEPS:
→ Build graph representation (PyTorch Geometric Data object)
→ Implement baseline models (Logistic Regression, XGBoost)
→ Implement GraphSAGE
→ Compare results
""")

print("=" * 80)
print("✓ EDA COMPLETE - Ready for modeling!")
print("=" * 80)

LOADING ELLIPTIC DATASET
Loading Elliptic dataset...
✓ Features: (203769, 167) (nodes x features)
✓ Edges: (234355, 2)
✓ Classes: (203769, 2)

✓ Data loaded successfully!

DATA STRUCTURE INSPECTION

1. FEATURES DATAFRAME
   Shape: (203769, 167)
   Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]... (showing first 10)
   Memory usage: 272.24 MB

   First few rows:
         0    1         2         3         4        5         6         7    \
0  230425980    1 -0.171469 -0.184668 -1.201369 -0.12197 -0.043875 -0.113002   
1    5530458    1 -0.171484 -0.184668 -1.201369 -0.12197 -0.043875 -0.113002   
2  232022460    1 -0.172107 -0.184668 -1.201369 -0.12197 -0.043875 -0.113002   

        8         9    ...       157       158       159       160       161  \
0 -0.061584 -0.162097  ... -0.562153 -0.600999  1.461330  1.461369  0.018279   
1 -0.061584 -0.162112  ...  0.947382  0.673103 -0.979074 -0.978556  0.018279   
2 -0.061584 -0.162749  ...  0.670883  0.439728 -0.979074 -0.978556 -0.098889   

 

ValueError: Length mismatch: Expected axis has 167 elements, new values have 168 elements